In [1]:
import numpy as np
import pandas as pd
import re
import time
import pickle
import pprint
from glob import glob

from selenium import webdriver

from hackaroutine import Ingredient, Product, Routine

In [2]:
# CHROMEDRIVER_PATH = '/Users/Jelly/Documents/googledrive/chromedriver/macos/chromedriver'
CHROMEDRIVER_PATH = 'C:/chromedriver_win32/chromedriver.exe'

In [ ]:
driver = webdriver.Chrome(CHROMEDRIVER_PATH)

In [3]:
file_list = glob('./data/*.csv')
file_list

['./data\\brands.csv',
 './data\\recommendations_2020-05-05.csv',
 './data\\responses_2020-05-05.csv',
 './data\\responses_modified_2020-05-05.csv']

In [4]:
responses = pd.read_csv(file_list[3])
new_columns = [
    'timestamp',
    'skin_type',
    'cannot_contain',
    'skin_concern',
    'am_routine',
    'pm_routine',
    'climate',
    'skin_sensitivities',
    'used_retinoids',
    'used_acids',
    'prone_to_breakouts',
    'miscellaneous',
    'permission'
]
responses.columns = new_columns
responses.head(2)

,timestamp,skin_type,cannot_contain,skin_concern,am_routine,pm_routine,climate,skin_sensitivities,used_retinoids,used_acids,prone_to_breakouts,miscellaneous,permission
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes


In [5]:
routine_columns = ['am_routine', 'pm_routine']

for col in routine_columns:
    responses[col] = responses[col].str.lower().str.replace(';', ',')\
                     .str.replace('\n', ',').str.split(r'\,+\s*')
    print(responses[col][:2])

0    [trader joe's all in one cleanser, bioderma se...
1    [biore charcoal acne clearing cleanser, kiel's...
Name: am_routine, dtype: object
0    [trader joe's all in one cleanser, bioderma se...
1                                                  NaN
Name: pm_routine, dtype: object


In [6]:
responses['am_routine'][:5]

0    [trader joe's all in one cleanser, bioderma se...
1    [biore charcoal acne clearing cleanser, kiel's...
2    [drunk elephant beste no. 9 jelly cleanser, pa...
3    [paula's choice enriched calming toner, timele...
4    [osea ocean cleanser, renee rouleau elderberry...
Name: am_routine, dtype: object

In [7]:
def routine_analyzer(routine):
    print(f'{len(routines)} routines')
    start = int(input('start:'))
    end = int(input('end:'))
    routine_objects = []
    cosdna_names = []
    for i in range(start - 1, end):
        print(f'{i + 1} of {end} ({len(routines)})')
        text = input('proceed?')
        if text == 'break':
            break
        elif text == 'skip':
            continue
        else:
            routine = Routine(name=i, routine=routines[i])
            routine.link_sync()
            routine_objects.append(routine)
            cosdna_names.append(routine.routine)
    return routine_objects, cosdna_names

In [8]:
with open('./data/am_routine_objects.pickle', 'rb') as handle:
    am_routine_objects = pickle.load(handle)

with open('./data/am_cosdna_names.pickle', 'rb') as handle:
    am_cosdna_names = pickle.load(handle)

In [9]:
am_cosdna_names[:5]

[["trader joe's all in one cleanser",
  'bioderma sébium h2o water (up-to-date)!',
  'the ordinary ascorbic acid 8%',
  'bioderma sébium mat control',
  'purlisse bb cream'],
 ['biore charcoal acne clearing cleanser',
  "kiehl's powerful-strength line-reducing concentrate 12.5% vitamin c",
  'neutrogena rapid clear stubborn acne spot gel',
  'roc retinol correxion',
  'body merry retinol surge moisturizer'],
 ['drunk elephant beste no. 9 jelly cleanser',
  "paula's choice skin balancing pore-reducing toner",
  "paula's choice boost 10% niacinamide booster",
  "paula's choice boost 10% azelaic acid booster",
  'cerave pm facial moisturizing lotion',
  'eltamd uv clear broad-spectrum spf 46'],
 ["paula's choice skin recovery enriched calming toner",
  'timeless 20% vitamin c serum',
  'the ordinary niacinamide',
  'the ordinary hyaluronic acid',
  'cerave am facial moisturizing lotion spf 30',
  'cerave hydrating sunscreen spf 30 sheer tint '],
 ['osea ocean cleanser',
  'renee rouleau e

In [10]:
routines = responses['am_routine']

In [ ]:
print(f'{len(routines)} routines')

start = int(input('start:'))
end = int(input('end:'))

for i in range(start - 1, end):
    print(f'{i + 1} of {end} ({len(routines)})')
    text = input('proceed?')
    if text == 'break':
        break
    elif text == 'skip':
        continue
    else:
        routine = Routine(name=i, routine=routines[i])
        routine.link_sync()
        am_routine_objects.append(routine)
        am_cosdna_names.append(routine.routine)

In [11]:
routines[12]

['papaya & vitamin c revive & brighten bar soap',
 '',
 'stridex single-step acne contro maximum alcohol free',
 'timeless vitamin c',
 'mad hippie vitamin c serum',
 'kikumasamune high moist lotion',
 'hada labo gokujyun premium hyaluronic acid lotion',
 'heritage products rosewater and glycerin',
 'unscented vanicream moisturizing skin cream',
 'australian gold botanical spf 50 tinted face mineral lotion']

In [12]:
twelve = Routine(routine=routines[12])
twelve.link_sync()

No results for papaya & vitamin c revive & brighten bar soap on CosDNA.
Enter new search (to skip search, enter 'SKIP' w/o quotes):
No results for stridex single-step acne contro maximum alcohol free on CosDNA.
Enter new search (to skip search, enter 'SKIP' w/o quotes):
No results for heritage products rosewater and glycerin on CosDNA.
Enter new search (to skip search, enter 'SKIP' w/o quotes):


In [13]:
twelve.routine

['papaya & vitamin c revive & brighten bar soap',
 'stridex maximum alcohol free',
 'timeless vitamin c',
 'mad hippie vitamin c serum',
 'kikumasamune high moist lotion',
 'hada-labo gokujyun premium hyaluronic acid lotion',
 'heritage rosewater',
 'unscented vanicream moisturizing skin cream',
 'australian gold botanical spf 50 tinted face mineral lotion']

In [19]:
twelve.products[1].ingredients

['ammonium lauryl sulfate',
 'ammonium xylenesulfonate',
 'citric acid',
 'dmdm hydantoin',
 'fragrance',
 'menthol',
 'ppg-5-ceteth-20',
 'purified water',
 'simethicone',
 'sodium borate',
 'tetrasodium edta']

In [20]:
stridex = Product('stridex maximum alcohol free')
stridex.link_sync()

In [21]:
stridex.ingredients

['ammonium lauryl sulfate',
 'ammonium xylenesulfonate',
 'citric acid',
 'dmdm hydantoin',
 'fragrance',
 'menthol',
 'ppg-5-ceteth-20',
 'purified water',
 'simethicone',
 'sodium borate',
 'tetrasodium edta']

In [ ]:
papaya = Product('papaya & vitamin c revive & brighten bar soap')

In [ ]:
papaya.link_sync()

In [ ]:
papaya._skip

In [ ]:
am_routine_objects

In [ ]:
# resume at 13 after debugging

with open('./data/am_routine_objects.pickle', 'wb') as handle:
    pickle.dump(am_routine_objects, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./data/am_cosdna_names.pickle', 'wb') as handle:
    pickle.dump(am_cosdna_names, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# big_list = []
# for routine in responses['am_routine']:
#     for product in routine:
#         big_list.append(product)

In [ ]:
# prior experience

popular_brands = [
    'La Roche Posay',
    'Cerave', 
    'Cetaphil', 
    'Timeless Skincare', 
    'Skinceuticals', 
    'Stratia Skincare', 
    'Paula’s Choice', 
    'CosRX', 
    'The Ordinary', 
    'Drunk Elephant', 
    'Klairs', 
    'Inkey List', 
    'Farmacy ', 
    'First Aid Beauty'
]

popular_ingredients = [
    'Vitamin C', 
    'Glycolic Acid', 
    'Lactic Acid', 
    'Mandelic Acid', 
    'Niacinamide', 
    'Retinol', 
    'Hydroxypinocolone Retinoate', 
    'Bakuchiol', 
    'Retinyl Palmitate', 
    'Ceramides', 
    'Salicylic Acid', 
    'Willow bark Extract', 
    'Adapalene', 
    'Sodium Lauryl Sulfate', 
    'Cocoamidopropyl Betaine', 
]

In [ ]:
tick = time.time()
n = 3
routine = Routine(name=n, routine=responses['am_routine'][n])
routine.link_sync()
print(routine.routine)
tock = time.time()
print(f'{tock - tick} s')

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer(sparse=True)
features = dv.fit_transform(dict(routine.top_ingredients()))

In [ ]:
routine.has('1,3-butanediol')

In [ ]:
routine.routine

In [ ]:
for ingredient in popular_ingredients:
    print(ingredient)
    print(routine.has(ingredient))
    print()

## string grouper demo

string_grouper is giving me a hard time, so i am going to do this manually according to:
https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [ ]:
import json
from string_grouper import match_strings, match_most_similar, group_similar_strings, StringGrouper

In [ ]:
with open('./data/big_dict.json', 'rb') as handle:
    big_dict = json.load(handle)

In [ ]:
product_list = pd.read_json('./data/big_dict.json', orient='index').reset_index(level=0)
product_list.columns = ['name', 'brand', 'product']
product_list.head()

In [ ]:
product_list['name']

In [ ]:
# matches = match_most_similar(product_list['name'], sample_list)
# pd.DataFrame({'sample_list': sample_list, 'matches': matches})

In [ ]:
import re
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
# def ngrams(string, n=3):
#     string = re.sub(r'[,-./]|\s',r'', string)
#     ngrams = zip(*[string[i:] for i in range(n)])
#     return [''.join(ngram) for ngram in ngrams]

def ngrams(string, n=3):
    string = unidecode.unidecode(string)
    string = string.lower()
    chars_to_remove = [")", "(", ".", "|", "[", "]", "{", "}", "'", '"', 
                       "?", "!"]
    punctuation = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(punctuation, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = re.sub(' +',' ',string).strip()
    string = ' '*(n-1) + string +' '*(n-1) # pad names for ngrams...
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

# vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
# matrix = vectorizer.fit_transform(sample_list)

In [ ]:
ngrams('Lancôme')

In [ ]:
print(matrix[0])
print(ngrams(sample_list[0]))

In [ ]:
# https://caserta.com/data-blog/string-matching-record-linkage-python-strategies/

def tfidf_match(list1, list2, sort=False):
    """For each item in list1, find the match in list2"""

    list1c = list1.copy()
    list2c = list2.copy()

    def split_sorter(name):
        split = re.split(' +', name)
        split.sort()
        return ' '.join(split)

    if sort:
        list1c = [split_sorter(e) for e in list1c]
        list2c = [split_sorter(e) for e in list2c]
            
    vectorizer = TfidfVectorizer(analyzer=ngrams, lowercase=True)
    tfidf = vectorizer.fit_transform(list2c)
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
    distances, indices = nbrs.kneighbors(vectorizer.transform(list1c))
    
    matches = [(round(distances[i][0], 2), list1[i], list2[j[0]]) 
               for i, j in enumerate(indices)]
    matches = pd.DataFrame(matches, 
                           columns=['distance', 'input', 'matched'])
    return matches

In [ ]:
def split_sorter(name):
    split = re.split(' +', name)
    split.sort()
    return ' '.join(split)

In [ ]:
split_sorter("kiehl's vitamin c serum powerful-strength")

In [ ]:
tick = time.time()
kiehls = [
    "Kiel's Powerful-Strength Line-Reducing Concentrate 12.5% Vitamin C Serum",
    "kiehl's vitamin c serum powerful-strength",
    "Kiehl's Calendula hydrating mask 5 minute",
    "Kiehl's Blue Herbal Acne Cleansing Treatment",
    "Kiehl's ultrafacial toner"
]
matches = tfidf_match(kiehls, product_list['name'], sort=True)
tock = time.time()
print(f'{tock - tick} s')
matches

In [ ]:
split = kiehls[0].split(' ')
split.sort()

In [ ]:
split

In [ ]:
lrp = [
    "La Roche-Posay tinted Anthelios 50 Mineral Sunscreen",
    "La Roche Posay Medicated Gel Cleanser",
    "La Roche-Posay Tolerane makeup remover"
]
tfidf_match(lrp, product_list['name'])

In [ ]:
english = [
    'first i use la roche posay effaclare cleanser', 
    'then moisturize using kiehls super multi corrective cream',
    'then kiehls facial fuel sunblock if i go outside']

tfidf_match(english, product_list['name'], sort=True)

## using rapidfuzz

In [ ]:
from rapidfuzz import fuzz
from rapidfuzz import process

In [ ]:
process.extractOne(
    "La Roche-Posay Tolerane makeup remover", 
    product_list['name']
    )

## common ingredient dictionary

In [ ]:
with open('./data/master_dict.pickle', 'rb') as handle:
    master_dict = pickle.load(handle)

In [ ]:
def ingredient_dict_builder(routine_dict, master_dict):
    base_url = 'https://cosdna.com/eng/'
    filtered_dict = {k:v for k,v in routine_dict.items() if k not in master_dict.keys()}
    for k, v in filtered_dict.items():
        driver.get(base_url + k + '.html')
        print(f'{k}: {v}')
        ingredient = input('preferred name: ')
        if ingredient == 'BREAK':
            break
        elif ingredient == '':
            master_dict[k] = v
        else:
            master_dict[k] = ingredient
    return master_dict

In [ ]:
am_routines = []
for rout in responses['am_routine']:
    routine = Routine(name=i, routine=responses['am_routine'][i])
    routine.link_sync()
#     master_dict = ingredient_dict_builder(routine._routine_dict, master_dict)
    am_routines.append(routine)

In [ ]:
for i in range(5, 10):
    routine = Routine(name=i, routine=responses['am_routine'][i])
    routine.link_sync()
    master_dict= ingredient_dict_builder(routine._routine_dict, master_dict)

In [ ]:
master_dict

In [ ]:
# saving pickle
with open('./data/master_dict.pickle', 'wb') as handle:
    pickle.dump(master_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)